# Introduction

In this notebook, we will do a comprehensive analysis of the Android app market by comparing thousands of apps in the Google Play store.

Things given: Notebook structure, challenges, and reading the csv initially with pandas

In today's task we will analyze the data about apps on the Google Play Store, and create a sort of an analytics, using plotly.

This will give us several insights: Competition in the play store, most popular app category, paid vs free apps and price sweet spot, whether paid apps have a good ROI.

Apparently today I'll learn:
1. Remove duplicates, remove unwanted symbols to make data numeric
2. Working with columns containing nested data
3. Data visualizations using plotly library: bar charts, pie charts, donut charts, scatter plots and color scales

# About the Dataset of Google Play Store Apps & Reviews

**Data Source:** <br>
App and review data was scraped from the Google Play Store by Lavanya Gupta in 2018. Original files listed [here](
https://www.kaggle.com/lava18/google-play-store-apps).

# Import Statements

In [62]:
import pandas as pd
import plotly.express as px

# Notebook Presentation

In [2]:
# Show numeric output in decimal format e.g., 2.15
pd.options.display.float_format = '{:,.2f}'.format

# Read the Dataset

In [3]:
df_apps = pd.read_csv('apps.csv')

# Data Cleaning

**Challenge**: How many rows and columns does `df_apps` have? What are the column names? Look at a random sample of 5 different rows with [.sample()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.sample.html).

In [5]:
df_apps.shape

(10841, 12)

In [6]:
df_apps.columns

Index(['App', 'Category', 'Rating', 'Reviews', 'Size_MBs', 'Installs', 'Type',
       'Price', 'Content_Rating', 'Genres', 'Last_Updated', 'Android_Ver'],
      dtype='object')

In [8]:
df_apps.sample(5)

,App,Category,Rating,Reviews,Size_MBs,Installs,Type,Price,Content_Rating,Genres,Last_Updated,Android_Ver
2493,Fodbold DK Pro,SPORTS,4.60,315,4.30,"1,000",Paid,$2.99,Everyone,Sports,"June 6, 2018",4.0 and up
3733,Florida Today,NEWS_AND_MAGAZINES,3.30,202,38.00,"10,000",Free,0,Everyone 10+,News & Magazines,"June 20, 2018",5.0 and up
9329,MakeMyTrip-Flight Hotel Bus Cab IRCTC Rail Boo...,TRAVEL_AND_LOCAL,4.30,599872,12.00,"10,000,000",Free,0,Everyone,Travel & Local,"August 4, 2018",4.4 and up
709,MY GULFPORT FL,PRODUCTIVITY,NaN,0,49.00,50,Free,0,Everyone,Productivity,"May 18, 2018",4.1 and up
10205,Block Craft 3D: Building Simulator Games For Free,GAME,4.50,946926,57.00,"50,000,000",Free,0,Everyone,Simulation,"March 5, 2018",4.0.3 and up


Simple enough, giving no parameter gives us a single random row

By looking at this data, we can see some issues we might need to fix: the NaN values for Rating for some apps, and the $ symbol in front of price which doesn't allow numerical operations on it.

### Drop Unused Columns

**Challenge**: Remove the columns called `Last_Updated` and `Android_Version` from the DataFrame. We will not use these columns.

In [22]:
# df_apps.head()
df_apps.drop(columns=['Last_Updated', 'Android_Ver'], inplace=True)

I just used my intuition for this one, looked for anything that said "drop/delete/remove/column" but ended up experimenting with this and figuring out.

Initially I hadn't used inplace, so the actual dataframe wasn't updated. I only found that out when I was working with df_apps_clean below.

In [30]:
# Course solution:
# df_apps.drop(['Last_Updated', 'Android_Ver'], axis=1, inplace=True)

### Find and Remove NaN values in Ratings

**Challenge**: How may rows have a NaN value (not-a-number) in the Ratings column? Create DataFrame called `df_apps_clean` that does not include these rows.

In [23]:
df_apps.Rating.isna().value_counts()

,count
Rating,
False,9367
True,1474


The course solution had a different way to look for the NaN values, by creating a whole another dataframe with those values. But I feel like I didn't use the filtering ability well so here's that method as well:

In [32]:
nan_rows = df_apps[df_apps.Rating.isna()]
print(nan_rows.shape)
nan_rows.head()

(1474, 10)


,App,Category,Rating,Reviews,Size_MBs,Installs,Type,Price,Content_Rating,Genres
0,Ak Parti Yardım Toplama,SOCIAL,NaN,0,8.70,0,Paid,$13.99,Teen,Social
1,Ain Arabic Kids Alif Ba ta,FAMILY,NaN,0,33.00,0,Paid,$2.99,Everyone,Education
2,Popsicle Launcher for Android P 9.0 launcher,PERSONALIZATION,NaN,0,5.50,0,Paid,$1.49,Everyone,Personalization
3,Command & Conquer: Rivals,FAMILY,NaN,0,19.00,0,NaN,0,Everyone 10+,Strategy
4,CX Network,BUSINESS,NaN,0,10.00,0,Free,0,Everyone,Business


Looking at this, I think I need to do some more digging. I want to find out how many of these NaN rated apps have 0 reviews and how many have 0 installs

In [35]:
print(nan_rows[nan_rows.Reviews == 0].Reviews.count())
nan_rows.sort_values(by='Reviews', ascending=False)

596


,App,Category,Rating,Reviews,Size_MBs,Installs,Type,Price,Content_Rating,Genres
5428,We learn children's verses in kindergarten,PARENTING,NaN,3248,6.50,"100,000",Free,0,Everyone,Parenting;Education
5365,Adivina el Emoji,GAME,NaN,2536,32.00,"100,000",Free,0,Everyone,Trivia
6141,Young Speeches,LIBRARIES_AND_DEMO,NaN,2221,2.40,"500,000",Free,0,Everyone,Libraries & Demo
4558,【Ranobbe complete free】 Novelba - Free app tha...,COMICS,NaN,1330,22.00,"50,000",Free,0,Everyone,Comics
5840,Em Fuga Brasil,FAMILY,NaN,1317,60.00,"100,000",Free,0,Everyone,Simulation
...,...,...,...,...,...,...,...,...,...,...
9,Eu sou Rico,FINANCE,NaN,0,2.60,0,Paid,$30.99,Everyone,Finance
8,cronometra-br,PRODUCTIVITY,NaN,0,5.40,0,Paid,$154.99,Everyone,Productivity
7,EG | Explore Folegandros,TRAVEL_AND_LOCAL,NaN,0,56.00,0,Paid,$3.99,Everyone,Travel & Local
6,Pekalongan CJ,SOCIAL,NaN,0,5.90,0,Free,0,Teen,Social


We can see that there are some apps with 1k+ reviews and yet no data for rating. Interesting.

Now let's dig in similarly for install numbers

In [40]:
print(nan_rows[nan_rows.Installs == '0'].Installs.count())
# Apparently the comma makes it a string
# nan_rows.sort_values(by='Installs', ascending=False)
# Since the value is not a number, this won't work so we need to convert into string first
# After looking around, this was the most appropriate way I found for right now:
nan_rows['Installs'] = nan_rows['Installs'].str.replace(',','').astype(float)
nan_rows.sort_values(by='Installs', ascending=False)

15


<ipython-input-40-f0294c6bf2b7>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nan_rows['Installs'] = nan_rows['Installs'].str.replace(',','').astype(float)


,App,Category,Rating,Reviews,Size_MBs,Installs,Type,Price,Content_Rating,Genres
7175,Android TV Remote Service,TOOLS,NaN,1,3.70,"1,000,000.00",Free,0,Everyone,Tools
7035,SD card backup,TOOLS,NaN,142,3.40,"1,000,000.00",Free,0,Everyone,Tools
6141,Young Speeches,LIBRARIES_AND_DEMO,NaN,2221,2.40,"500,000.00",Free,0,Everyone,Libraries & Demo
4803,LH Housing Promotion Notice,HOUSE_AND_HOME,NaN,160,3.10,"100,000.00",Free,0,Everyone,House & Home
5004,Ai La Trieu Phu - ALTP Free,EDUCATION,NaN,776,24.00,"100,000.00",Free,0,Everyone,Education
...,...,...,...,...,...,...,...,...,...,...
4,CX Network,BUSINESS,NaN,0,10.00,0.00,Free,0,Everyone,Business
5,Test Application DT 02,ART_AND_DESIGN,NaN,0,1.20,0.00,Free,0,Everyone,Art & Design
6,Pekalongan CJ,SOCIAL,NaN,0,5.90,0.00,Free,0,Teen,Social
7,EG | Explore Folegandros,TRAVEL_AND_LOCAL,NaN,0,56.00,0.00,Paid,$3.99,Everyone,Travel & Local


Okay so apparently Android TV Remote Service has no rating despire 1M installs, understandable. Others seem to be niche as well, but how they have 100k+ downloads is beyond me.

In [25]:
df_apps_clean = df_apps.dropna(subset='Rating')

I didn't need to use the subset parameter but I had to because when I tested it said there were 2 other rows that had an NaN value, but turns out that was because I hadn't actually deleted Android_Ver column yet. So it works well either way, but I'm keeping it there as an artifact.

### Find and Remove Duplicates

**Challenge**: Are there any duplicates in data? Check for duplicates using the [.duplicated()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.duplicated.html) function. How many entries can you find for the "Instagram" app? Use [.drop_duplicates()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop_duplicates.html) to remove any duplicates from `df_apps_clean`.


In [28]:
df_apps_clean.duplicated().value_counts()

,count
False,8891
True,476


From course solution: Yes I forgot that this scalar of booleans only can be used to filter a dataframe by rows

In [41]:
df_apps_clean[df_apps_clean.duplicated()]

,App,Category,Rating,Reviews,Size_MBs,Installs,Type,Price,Content_Rating,Genres
946,420 BZ Budeze Delivery,MEDICAL,5.00,2,11.00,100,Free,0,Mature 17+,Medical
1133,MouseMingle,DATING,2.70,3,3.90,100,Free,0,Mature 17+,Dating
1196,"Cardiac diagnosis (heart rate, arrhythmia)",MEDICAL,4.40,8,6.50,100,Paid,$12.99,Everyone,Medical
1231,Sway Medical,MEDICAL,5.00,3,22.00,100,Free,0,Everyone,Medical
1247,Chat Kids - Chat Room For Kids,DATING,4.70,6,4.90,100,Free,0,Mature 17+,Dating
...,...,...,...,...,...,...,...,...,...,...
10802,Skype - free IM & video calls,COMMUNICATION,4.10,10484169,3.50,"1,000,000,000",Free,0,Everyone,Communication
10809,Instagram,SOCIAL,4.50,66577313,5.30,"1,000,000,000",Free,0,Teen,Social
10826,Google Drive,PRODUCTIVITY,4.40,2731211,4.00,"1,000,000,000",Free,0,Everyone,Productivity
10832,Google News,NEWS_AND_MAGAZINES,3.90,877635,13.00,"1,000,000,000",Free,0,Teen,News & Magazines


I had used filtering below but I think I can just list the rows straight up instead

In [44]:
print(df_apps_clean[df_apps_clean.App == 'Instagram'].duplicated().value_counts())
df_apps_clean[df_apps_clean.App == 'Instagram'][df_apps_clean.duplicated()]

False    3
True     1
Name: count, dtype: int64


<ipython-input-44-8dca49ac76bc>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_apps_clean[df_apps_clean.App == 'Instagram'][df_apps_clean.duplicated()]


,App,Category,Rating,Reviews,Size_MBs,Installs,Type,Price,Content_Rating,Genres
10809,Instagram,SOCIAL,4.50,66577313,5.30,"1,000,000,000",Free,0,Teen,Social


I tried to use 'and' to join the two conditions but we can't do that on scalars, so I had to settle for using both the filters one after another. So now I have the row(s) with App=Instagram, and among them, the rows that are duplicates.

In the course, we first look at all the rows with the name Instagram and then we decide we want to delete all, even though they have a different value for Reviews. Let's first look at it.

In [45]:
df_apps_clean[df_apps_clean.App == 'Instagram']

,App,Category,Rating,Reviews,Size_MBs,Installs,Type,Price,Content_Rating,Genres
10806,Instagram,SOCIAL,4.50,66577313,5.30,"1,000,000,000",Free,0,Teen,Social
10808,Instagram,SOCIAL,4.50,66577446,5.30,"1,000,000,000",Free,0,Teen,Social
10809,Instagram,SOCIAL,4.50,66577313,5.30,"1,000,000,000",Free,0,Teen,Social
10810,Instagram,SOCIAL,4.50,66509917,5.30,"1,000,000,000",Free,0,Teen,Social


But just using drop_duplicates won't work because it will not delete unless two rows are completely identical, so it will delete only one of the above four rows. So we need to set the columns which need to be looked at in order to determine whether two rows are duplicates.

In [50]:
df_apps_clean.drop_duplicates(subset=['App', 'Type', 'Price'], inplace=True)

<ipython-input-50-37fcf962bced>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_apps_clean.drop_duplicates(subset=['App', 'Type', 'Price'], inplace=True)


In [51]:
print(df_apps_clean.shape)
df_apps_clean.head()

(8199, 10)


,App,Category,Rating,Reviews,Size_MBs,Installs,Type,Price,Content_Rating,Genres
21,KBA-EZ Health Guide,MEDICAL,5.00,4,25.00,1,Free,0,Everyone,Medical
28,Ra Ga Ba,GAME,5.00,2,20.00,1,Paid,$1.49,Everyone,Arcade
47,Mu.F.O.,GAME,5.00,2,16.00,1,Paid,$0.99,Everyone,Arcade
82,Brick Breaker BR,GAME,5.00,7,19.00,5,Free,0,Everyone,Arcade
99,Anatomy & Physiology Vocabulary Exam Review App,MEDICAL,5.00,1,4.60,5,Free,0,Everyone,Medical


Okay now we've gone from 9367 entries to 8199.

This number is not a complete representation of the Play Store but a sample, but we'll take it as a representation for now. In actuality the apps here may have different prices, may be different depending on geography and time at scraping.

Angela had already cleaned the data a bit by making app size floats, removing the + on install count, etc.

# Find Highest Rated Apps

**Challenge**: Identify which apps are the highest rated. What problem might you encounter if you rely exclusively on ratings alone to determine the quality of an app?

In [56]:
df_apps_clean.sort_values(by='Rating', ascending=False).head()

,App,Category,Rating,Reviews,Size_MBs,Installs,Type,Price,Content_Rating,Genres
21,KBA-EZ Health Guide,MEDICAL,5.00,4,25.00,1,Free,0,Everyone,Medical
1751,Spine- The dating app,DATING,5.00,5,9.30,500,Free,0,Teen,Dating
1761,Selfie With Champion AJ Style,PHOTOGRAPHY,5.00,2,7.50,500,Free,0,Everyone,Photography
1763,Ag Valley Cooperative,BUSINESS,5.00,6,74.00,500,Free,0,Everyone,Business
1769,Yazdani Cd Center EllahAbad Official App,FAMILY,5.00,8,3.80,500,Free,0,Everyone,Entertainment


Straight off the bat, the highest rated apps are those with very few reviews and very few installs. This doesn't tell us much because their sample is very low and not enough people have decided on its true rating.

# Find 5 Largest Apps in terms of Size (MBs)

**Challenge**: What's the size in megabytes (MB) of the largest Android apps in the Google Play Store. Based on the data, do you think there could be limit in place or can developers make apps as large as they please?

In [55]:
df_apps_clean.sort_values(by='Size_MBs', ascending=False).head()

,App,Category,Rating,Reviews,Size_MBs,Installs,Type,Price,Content_Rating,Genres
8719,Draft Simulator for FUT 18,SPORTS,4.60,162933,100.00,"5,000,000",Free,0,Everyone,Sports
8718,Mini Golf King - Multiplayer Game,GAME,4.50,531458,100.00,"5,000,000",Free,0,Everyone,Sports
10295,SimCity BuildIt,FAMILY,4.50,4218587,100.00,"50,000,000",Free,0,Everyone 10+,Simulation
9945,Ultimate Tennis,SPORTS,4.30,183004,100.00,"10,000,000",Free,0,Everyone,Sports
9944,Gangster Town: Vice District,FAMILY,4.30,65146,100.00,"10,000,000",Free,0,Mature 17+,Simulation


Well this doesn't seem to be an accurate representation of the Play Store, but anyway, from this we can see that there's a limit of 100MB placed on the app size. All the top 5 apps based on size here are games, even though the category says otherwise. Doesn't look like a very good dataset imho.

# Find the 5 App with Most Reviews

**Challenge**: Which apps have the highest number of reviews? Are there any paid apps among the top 50?

In [60]:
df_apps_clean.sort_values(by='Reviews', ascending=False).head(50)

,App,Category,Rating,Reviews,Size_MBs,Installs,Type,Price,Content_Rating,Genres
10805,Facebook,SOCIAL,4.10,78158306,5.30,"1,000,000,000",Free,0,Teen,Social
10785,WhatsApp Messenger,COMMUNICATION,4.40,69119316,3.50,"1,000,000,000",Free,0,Everyone,Communication
10806,Instagram,SOCIAL,4.50,66577313,5.30,"1,000,000,000",Free,0,Teen,Social
10784,Messenger – Text and Video Chat for Free,COMMUNICATION,4.00,56642847,3.50,"1,000,000,000",Free,0,Everyone,Communication
10650,Clash of Clans,GAME,4.60,44891723,98.00,"100,000,000",Free,0,Everyone 10+,Strategy
10744,Clean Master- Space Cleaner & Antivirus,TOOLS,4.70,42916526,3.40,"500,000,000",Free,0,Everyone,Tools
10835,Subway Surfers,GAME,4.50,27722264,76.00,"1,000,000,000",Free,0,Everyone 10+,Arcade
10828,YouTube,VIDEO_PLAYERS,4.30,25655305,4.65,"1,000,000,000",Free,0,Teen,Video Players & Editors
10746,"Security Master - Antivirus, VPN, AppLock, Boo...",TOOLS,4.70,24900999,3.40,"500,000,000",Free,0,Everyone,Tools
10584,Clash Royale,GAME,4.60,23133508,97.00,"100,000,000",Free,0,Everyone 10+,Strategy


I can use head(50) to list the top 50 apps here like the course does, but since we want to know whether there's any paid apps among top 50, I think I'll just do a filter as well to confirm.

In [59]:
df_apps_clean.sort_values(by='Reviews', ascending=False).head(50)[df_apps_clean.Type == 'Paid']

<ipython-input-59-c516fb3316a8>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_apps_clean.sort_values(by='Reviews', ascending=False).head(50)[df_apps_clean.Type == 'Paid']


,App,Category,Rating,Reviews,Size_MBs,Installs,Type,Price,Content_Rating,Genres


As we can see, there's none

# Plotly Pie and Donut Charts - Visualise Categorical Data: Content Ratings

In [63]:
ratings = df_apps_clean.Content_Rating.value_counts()
ratings

,count
Content_Rating,
Everyone,6621
Teen,912
Mature 17+,357
Everyone 10+,305
Adults only 18+,3
Unrated,1


We can find the number of apps in each category as above. Now we may want to plot it so we need to import the plotly.express module above, and then use it to plot the data.

In order to plot, we must create a Figure object then use show() on it

In [69]:
fig = px.pie(ratings.index, ratings.values)
fig.show()

I could also have used keyword arguments instead of positional arguments, the first one is called labels and second is values.
Since I'm using a dataframe with a single column i.e. just a straight map from index to value, maybe I can just give the dataframe itself as an input.
Ok I tried but it doesn't work, it shows an empty window.

Now I need to look at the <a href='https://plotly.com/python-api-reference/generated/plotly.express.pie.html'>Plotly Pie Documentation</a> to figure out how to customize the pie chart.

In [72]:
fig = px.pie(labels=ratings.index, names=ratings.index, values=ratings.values,
             title="Content Ratings")
fig.update_traces(textposition='outside', textinfo='percent+label')
fig.show()

The update traces function lets us change the things that can't be changed from within the px.pie method. I had to copy that line from the course because I wouldn't have known otherwise.

Now if we instead want to create a donut chart, we can do it by simply adding an argument for 'hole'. It determines the radius of the inner masking circle, it is basically the ratio of radii of inner to outer circle. 0(pie-chart)<hole<1(empty)

In [75]:
fig = px.pie(labels=ratings.index, names=ratings.index, values=ratings.values,
             title="Content Ratings",
             hole=0.6)
fig.update_traces(textposition='inside',textfont_size=15, textinfo='percent')
fig.show()

# Numeric Type Conversion: Examine the Number of Installs

**Challenge**: How many apps had over 1 billion (that's right - BILLION) installations? How many apps just had a single install?

Check the datatype of the Installs column.

Count the number of apps at each level of installations.

Convert the number of installations (the Installs column) to a numeric data type. Hint: this is a 2-step process. You'll have make sure you remove non-numeric characters first.

In [76]:
print(type(df_apps_clean.Installs))
# From course: I can use describe instead
df_apps_clean.Installs.describe()

<class 'pandas.core.series.Series'>


,Installs
count,8199
unique,19
top,"1,000,000"
freq,1417


To look at the data tpe I can use info on the dataframe as well

In [77]:
df_apps_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8199 entries, 21 to 10835
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   App             8199 non-null   object 
 1   Category        8199 non-null   object 
 2   Rating          8199 non-null   float64
 3   Reviews         8199 non-null   int64  
 4   Size_MBs        8199 non-null   float64
 5   Installs        8199 non-null   object 
 6   Type            8199 non-null   object 
 7   Price           8199 non-null   object 
 8   Content_Rating  8199 non-null   object 
 9   Genres          8199 non-null   object 
dtypes: float64(2), int64(1), object(7)
memory usage: 962.6+ KB


We can clearly see that Installs has a data type of object which means we can't do arithmetic operations here. I wonder how they want us to count number of apps at each installation level, probably using comparison with 1, 5, 10, 50, 100, 500, 1000, 5000 and so on...

In [83]:
df_apps_clean[['App', 'Installs']].groupby('Installs').count()

,App
Installs,
1,3
"1,000",698
"1,000,000",1417
"1,000,000,000",20
10,69
"10,000",988
"10,000,000",933
100,303
"100,000",1096


Oh how I was wrong, and stupid, forgetting we had a method to count each occurence. Anyway now we know what the values look like and our final job is to convert this into numbers, and since I had already looked around, the best way I found was to replace the , with nothing, then convert to float, but then again, I think I will need to look at the solution.

In [84]:
df_apps_clean.Installs = df_apps_clean.Installs.astype(str).str.replace(',','') # The same method
df_apps_clean.Installs = pd.to_numeric(df_apps_clean.Installs)
df_apps_clean[['App', 'Installs']].groupby('Installs').count()

<ipython-input-84-633a8853fe0a>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-84-633a8853fe0a>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,App
Installs,
1,3
5,9
10,69
50,56
100,303
500,199
1000,698
5000,425
10000,988


The steps seemed to be much the same, apart from the astype(string) used somehow, and the to_numeric instead of astype(int) or astype(float)

But anyhow, we got the answer. Most apps on the Play Store have in the range of 1M downloads apparently

# Find the Most Expensive Apps, Filter out the Junk, and Calculate a (ballpark) Sales Revenue Estimate

Let's examine the Price column more closely.

**Challenge**: Convert the price column to numeric data. Then investigate the top 20 most expensive apps in the dataset.

Remove all apps that cost more than $250 from the `df_apps_clean` DataFrame.

Add a column called 'Revenue_Estimate' to the DataFrame. This column should hold the price of the app times the number of installs. What are the top 10 highest grossing paid apps according to this estimate? Out of the top 10 highest grossing paid apps, how many are games?


In [86]:
df_apps_clean.Price = df_apps_clean.Price.astype(str).str.replace('$','')
df_apps_clean.Price = pd.to_numeric(df_apps_clean.Price)
df_apps_clean.head()

<ipython-input-86-d7f3e6082cb9>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-86-d7f3e6082cb9>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,App,Category,Rating,Reviews,Size_MBs,Installs,Type,Price,Content_Rating,Genres
21,KBA-EZ Health Guide,MEDICAL,5.00,4,25.00,1,Free,0.00,Everyone,Medical
28,Ra Ga Ba,GAME,5.00,2,20.00,1,Paid,1.49,Everyone,Arcade
47,Mu.F.O.,GAME,5.00,2,16.00,1,Paid,0.99,Everyone,Arcade
82,Brick Breaker BR,GAME,5.00,7,19.00,5,Free,0.00,Everyone,Arcade
99,Anatomy & Physiology Vocabulary Exam Review App,MEDICAL,5.00,1,4.60,5,Free,0.00,Everyone,Medical


Used much the same approach as the last one, just replacing the dollar sign with nothing then converting it to numeric type
Now let's see the top 20 most expensive ones

In [98]:
df_apps_clean.sort_values(by='Price', ascending=False).head(20)

,App,Category,Rating,Reviews,Size_MBs,Installs,Type,Price,Revenue_Estimate,Content_Rating,Genres
3946,I'm Rich - Trump Edition,LIFESTYLE,3.60,275,7.30,10000,Paid,400.00,"4,000,000.00",Everyone,Lifestyle
2394,I am Rich!,FINANCE,3.80,93,22.00,1000,Paid,399.99,"399,990.00",Everyone,Finance
2461,I AM RICH PRO PLUS,FINANCE,4.00,36,41.00,1000,Paid,399.99,"399,990.00",Everyone,Finance
1331,most expensive app (H),FAMILY,4.30,6,1.50,100,Paid,399.99,"39,999.00",Everyone,Entertainment
3145,I am rich(premium),FINANCE,3.50,472,0.94,5000,Paid,399.99,"1,999,950.00",Everyone,Finance
2775,I Am Rich Pro,FAMILY,4.40,201,2.70,5000,Paid,399.99,"1,999,950.00",Everyone,Entertainment
5765,I am rich,LIFESTYLE,3.80,3547,1.80,100000,Paid,399.99,"39,999,000.00",Everyone,Lifestyle
3221,I am Rich Plus,FAMILY,4.00,856,8.70,10000,Paid,399.99,"3,999,900.00",Everyone,Entertainment
3554,💎 I'm rich,LIFESTYLE,3.80,718,26.00,10000,Paid,399.99,"3,999,900.00",Everyone,Lifestyle
3114,I am Rich,FINANCE,4.30,180,3.80,5000,Paid,399.99,"1,999,950.00",Everyone,Finance


We can see that the price for troll apps are outrageous and they must be removed from the dataset before they spoil our revenue data because these apps have artificially inflated sales figures (think free sales and what not)

I actually had not removed them until the revenue calculation but felt that removal was necessary after I saw that these apps had ruined the top 10 list.

This output has a Revenue_Estimate column because it was run when the column was present, but before the filters were applied to remove apps above $250

### The most expensive apps sub $250

In [99]:
df_apps_clean = df_apps_clean[df_apps_clean.Price < 250]

Now that the apps are removed we can move towards looking at the more reasonably priced apps:

In [103]:
df_apps_clean.sort_values(by='Price', ascending=False).head()

,App,Category,Rating,Reviews,Size_MBs,Installs,Type,Price,Content_Rating,Genres
2281,Vargo Anesthesia Mega App,MEDICAL,4.60,92,32.00,1000,Paid,79.99,Everyone,Medical
1407,LTC AS Legal,MEDICAL,4.00,6,1.30,100,Paid,39.99,Everyone,Medical
2629,I am Rich Person,LIFESTYLE,4.20,134,1.80,1000,Paid,37.99,Everyone,Lifestyle
2481,A Manual of Acupuncture,MEDICAL,3.50,214,68.00,1000,Paid,33.99,Everyone,Medical
504,AP Art History Flashcards,FAMILY,5.00,1,96.00,10,Paid,29.99,Mature 17+,Education


### Highest Grossing Paid Apps (ballpark estimate)

In [104]:
revenue = df_apps_clean.Price * df_apps_clean.Installs
df_apps_clean.insert(loc=8, column="Revenue_Estimate", value=revenue)

To multiply, we might have instead used df_apps_clean.Price.mul(df_apps_clean.Installs)

And we could directly add a new column using df_apps_clean['Revenue_Estimate'] = the multiplied scalar

Now to get the top 10 highest grossing apps

In [105]:
df_apps_clean.sort_values(by='Revenue_Estimate', ascending=False).head(10)

,App,Category,Rating,Reviews,Size_MBs,Installs,Type,Price,Revenue_Estimate,Content_Rating,Genres
9220,Minecraft,FAMILY,4.50,2376564,19.00,10000000,Paid,6.99,"69,900,000.00",Everyone 10+,Arcade;Action & Adventure
8825,Hitman Sniper,GAME,4.60,408292,29.00,10000000,Paid,0.99,"9,900,000.00",Mature 17+,Action
7151,Grand Theft Auto: San Andreas,GAME,4.40,348962,26.00,1000000,Paid,6.99,"6,990,000.00",Mature 17+,Action
7477,Facetune - For Free,PHOTOGRAPHY,4.40,49553,48.00,1000000,Paid,5.99,"5,990,000.00",Everyone,Photography
7977,Sleep as Android Unlock,LIFESTYLE,4.50,23966,0.85,1000000,Paid,5.99,"5,990,000.00",Everyone,Lifestyle
6594,DraStic DS Emulator,GAME,4.60,87766,12.00,1000000,Paid,4.99,"4,990,000.00",Everyone,Action
6082,Weather Live,WEATHER,4.50,76593,4.75,500000,Paid,5.99,"2,995,000.00",Everyone,Weather
6746,Card Wars - Adventure Time,FAMILY,4.30,129603,23.00,1000000,Paid,2.99,"2,990,000.00",Everyone 10+,Card;Action & Adventure
7633,Five Nights at Freddy's,GAME,4.60,100805,50.00,1000000,Paid,2.99,"2,990,000.00",Teen,Action
7954,Bloons TD 5,FAMILY,4.60,190086,94.00,1000000,Paid,2.99,"2,990,000.00",Everyone,Strategy


I could further use a filter and a count function here but we can easily see that the number of games here is 4 by category, but in actuality, 7/10 are games.

# Plotly Bar Charts & Scatter Plots: Analysing App Categories

In [110]:
print(df_apps_clean.Category.nunique())
top10_category = df_apps_clean.Category.value_counts()[:10]
top10_category

33


,count
Category,
FAMILY,1606
GAME,910
TOOLS,719
PRODUCTIVITY,301
PERSONALIZATION,298
LIFESTYLE,297
FINANCE,296
MEDICAL,292
PHOTOGRAPHY,263


Now that we have the data about categories and number of apps in each category, we can plot them on a bar graph

### Vertical Bar Chart - Highest Competition (Number of Apps)

In [113]:
bar = px.bar(x= top10_category.index,
             y= top10_category.values)
bar.show()

Using positional arguments didn't give the desired output, so we should use positional arguments here

Just because there's many apps inside one category doesn't mean we shouldn't make an app in that category, maybe that category itself is popular.

So now we should look at the popularity of different categories

### Horizontal Bar Chart - Most Popular Categories (Highest Downloads)

In [114]:
popular_categories = df_apps_clean.groupby('Category').agg({'Installs': pd.Series.sum})
popular_categories.sort_values(by='Installs', ascending=False, inplace=True)
popular_categories

,Installs
Category,
GAME,13858762717
COMMUNICATION,11039241530
TOOLS,8099724500
PRODUCTIVITY,5788070180
SOCIAL,5487841475
PHOTOGRAPHY,4649143130
FAMILY,4437554490
VIDEO_PLAYERS,3916897200
TRAVEL_AND_LOCAL,2894859300


In [120]:
h_bar = px.bar(x= popular_categories.Installs,
               y= popular_categories.index,
               orientation='h',
               title= 'Category Popularity')
h_bar.update_layout(xaxis_title='Number of Downloads', yaxis_title='Category')
h_bar.show()

Added the title and orientation parameters from the course

### Category Concentration - Downloads vs. Competition

**Challenge**:
* First, create a DataFrame that has the number of apps in one column and the number of installs in another:

<img src=https://imgur.com/uQRSlXi.png width="350">

* Then use the [plotly express examples from the documentation](https://plotly.com/python/line-and-scatter/) alongside the [.scatter() API reference](https://plotly.com/python-api-reference/generated/plotly.express.scatter.html)to create scatter plot that looks like this.

<img src=https://imgur.com/cHsqh6a.png>

*Hint*: Use the size, hover_name and color parameters in .scatter(). To scale the yaxis, call .update_layout() and specify that the yaxis should be on a log-scale like so: yaxis=dict(type='log')

In [121]:
category_df = df_apps_clean.groupby('Category').agg({'App':pd.Series.count, 'Installs': pd.Series.sum})
category_df.head()

,App,Installs
Category,,
ART_AND_DESIGN,61,114233100
AUTO_AND_VEHICLES,73,53129800
BEAUTY,42,26916200
BOOKS_AND_REFERENCE,169,1665791655
BUSINESS,262,692018120


In [124]:
scatter = px.scatter(category_df,
                     x='App', y='Installs',
                     color='Installs', size='App',
                     hover_name=category_df.index)
scatter.update_layout(xaxis_title = "Number of Apps(Lower=More Concentrated)",
                      yaxis=dict(type='log'))
scatter.show()

In the very first attempt, yay

But the course solution was different, it created a df for only number of apps, then merged it with the one we already have for installation counts

I'll just paste the code here:

In [125]:
cat_number = df_apps_clean.groupby('Category').agg({'App': pd.Series.count})

In [126]:
cat_merged_df = pd.merge(cat_number, popular_categories, on='Category', how="inner")
print(f'The dimensions of the DataFrame are: {cat_merged_df.shape}')
cat_merged_df.sort_values('Installs', ascending=False)

The dimensions of the DataFrame are: (33, 2)


,App,Installs
Category,,
GAME,910,13858762717
COMMUNICATION,257,11039241530
TOOLS,719,8099724500
PRODUCTIVITY,301,5788070180
SOCIAL,203,5487841475
PHOTOGRAPHY,263,4649143130
FAMILY,1606,4437554490
VIDEO_PLAYERS,148,3916897200
TRAVEL_AND_LOCAL,187,2894859300


Finally the scatter plot, it's almost the same as mine

In [127]:
scatter = px.scatter(cat_merged_df, # data
                    x='App', # column name
                    y='Installs',
                    title='Category Concentration',
                    size='App',
                    hover_name=cat_merged_df.index,
                    color='Installs')

scatter.update_layout(xaxis_title="Number of Apps (Lower=More Concentrated)",
                      yaxis_title="Installs",
                      yaxis=dict(type='log'))

scatter.show()

# Extracting Nested Data from a Column

**Challenge**: How many different types of genres are there? Can an app belong to more than one genre? Check what happens when you use .value_counts() on a column with nested values? See if you can work around this problem by using the .split() function and the DataFrame's [.stack() method](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.stack.html).


In [129]:
print(len(df_apps_clean.Genres.unique()))

df_apps_clean.Genres.value_counts()

114


,count
Genres,
Tools,718
Entertainment,467
Education,429
Productivity,301
Personalization,298
...,...
Casual;Music & Video,1
Tools;Education,1
Role Playing;Brain Games,1


We can see that there are some entries here which consist of more than one genre, so we need to split them then stack onto a single column (had to glance at the course soolution)

In [139]:
all_genres = df_apps_clean.Genres.str.split(';', expand=True).stack() # didn't know expand parameter was available
print(len(all_genres.unique()))
genre_counts = all_genres.value_counts()
genre_counts

53


,count
Tools,719
Education,587
Entertainment,498
Action,304
Productivity,301
Personalization,298
Lifestyle,298
Finance,296
Medical,292
Sports,270


# Colour Scales in Plotly Charts - Competition in Genres

**Challenge**: Can you create this chart with the Series containing the genre data?

<img src=https://imgur.com/DbcoQli.png width=400>

Try experimenting with the built in colour scales in Plotly. You can find a full list [here](https://plotly.com/python/builtin-colorscales/).

* Find a way to set the colour scale using the color_continuous_scale parameter.
* Find a way to make the color axis disappear by using coloraxis_showscale.

In [150]:
genre_bar = px.bar(x= genre_counts.index[:15], y= genre_counts.values[:15],
                   color=genre_counts.values[:15], color_continuous_scale = 'Agsunset',
                   title='Top Genres')
genre_bar.update_layout(xaxis_title='Genre',
                        yaxis_title='Number of Apps',
                        coloraxis_showscale=False)
genre_bar.show()

Finally did it after much sturggle without having to look at the solution.

# Grouped Bar Charts: Free vs. Paid Apps per Category

In [151]:
df_apps_clean.Type.value_counts()

,count
Type,
Free,7595
Paid,589


We can see that a majority of the apps are free, but there's some paid apps as well. Let's look at these apps in detail by adding the category into the mix

In [153]:
free_vs_paid = df_apps_clean.groupby(['Category', 'Type'], as_index=False).agg({'App': pd.Series.count})
free_vs_paid.sample(5)

,Category,Type,App
29,HOUSE_AND_HOME,Free,62
51,SPORTS,Free,238
46,PRODUCTIVITY,Paid,18
24,FOOD_AND_DRINK,Paid,2
18,EVENTS,Free,45


By using as_index = False, we ensure the category does not become the index and stays as a column in the dataframe

**Challenge**: Use the plotly express bar [chart examples](https://plotly.com/python/bar-charts/#bar-chart-with-sorted-or-ordered-categories) and the [.bar() API reference](https://plotly.com/python-api-reference/generated/plotly.express.bar.html#plotly.express.bar) to create this bar chart:

<img src=https://imgur.com/LE0XCxA.png>

You'll want to use the `df_free_vs_paid` DataFrame that you created above that has the total number of free and paid apps per category.

See if you can figure out how to get the look above by changing the `categoryorder` to 'total descending' as outlined in the documentation here [here](https://plotly.com/python/categorical-axes/#automatically-sorting-categories-by-name-or-total-value).

In [162]:
fvp_bar = px.bar(x= free_vs_paid.Category, y= free_vs_paid.App,
                 title= 'Free vs Paid Apps by Category', color=free_vs_paid.Type,
                 barmode='group')
fvp_bar.update_layout(xaxis_title='Category', yaxis_title= 'Number of Apps',
                      xaxis={'categoryorder': 'total descending'},
                      yaxis=dict(type='log'))
fvp_bar.show()

I had done the plot but my bars were stacked, had to look at the solution for grouped bars and logarithmic y axis.

# Plotly Box Plots: Lost Downloads for Paid Apps

**Challenge**: Create a box plot that shows the number of Installs for free versus paid apps. How does the median number of installations compare? Is the difference large or small?

Use the [Box Plots Guide](https://plotly.com/python/box-plots/) and the [.box API reference](https://plotly.com/python-api-reference/generated/plotly.express.box.html) to create the following chart.

<img src=https://imgur.com/uVsECT3.png>


In [172]:
downloads_box = px.box(df_apps_clean, x='Type', color='Type', y='Installs',
                       notched=True, points='all',
                       title='How Many Downloads Are Paid Apps Giving Up?')
downloads_box.update_layout(yaxis=dict(type='log'))
downloads_box.show()

This thing looks wide af, but checking with the course soln, this seems to be what was done there as well.

# Plotly Box Plots: Revenue by App Category

**Challenge**: See if you can generate the chart below:

<img src=https://imgur.com/v4CiNqX.png>

Looking at the hover text, how much does the median app earn in the Tools category? If developing an Android app costs $30,000 or thereabouts, does the average photography app recoup its development costs?

Hint: I've used 'min ascending' to sort the categories.

In [183]:
paid_only = df_apps_clean[df_apps_clean.Type == 'Paid']
revenue_box = px.box(paid_only, x= 'Category', y= 'Revenue_Estimate',
                     title= 'How Much Can paid Apps Earn?')
revenue_box.update_layout(yaxis=dict(type='log'),
                          xaxis={'categoryorder':'min ascending'})

revenue_box.show()

Looks good to me. Tools has a median of 5990.

The average photography app makes about 20k so spending 30k would be risky, though if the app can push further, the q3 revenue is 250k+ so there's a decent opportunity.

Some categories also have a lot of outliers, like Game, Family, Personalization, Tools, Medical...

# How Much Can You Charge? Examine Paid App Pricing Strategies by Category

**Challenge**: What is the median price price for a paid app? Then compare pricing by category by creating another box plot. But this time examine the prices (instead of the revenue estimates) of the paid apps. I recommend using `{categoryorder':'max descending'}` to sort the categories.

In [184]:
paid_only.Price.median()

2.99

I had to look at the solution only to see a median method smh

In [185]:
price_box = px.box(paid_only, x='Category', y='Price',
                   title='How Much Can You Charge?')
price_box.update_layout(xaxis={'categoryorder': 'max descending'},
                        yaxis= dict(type='log'))
price_box.show()

Here we can see that Medical and Business Apps have higher than average median prices, while other categories like personalization have a lower median, with some outliers.

This is it for today's tasks. This took a long time.